In [2]:
import collections
import csv
import itertools
import json
import pandas as pd


Join duplicate Part Numbers and write to file

In [3]:
pn_and_qty_with_duplicates = pd.read_csv('data/PN_and_QTY_with_duplicates.csv')

In [10]:
pn_and_qty_no_duplicates = dict([(key, 0) for key in pn_and_qty['PN']])
pns = pn_and_qty['PN']
qtys = pn_and_qty['QTY']
for pn, qty in itertools.izip(pns, qtys):
    pn_and_qty_no_duplicates[pn] += qty

In [13]:
with open('data/PN_and_QTY.csv', 'wb') as f:
    w = csv.DictWriter(f, ['PN', 'QTY'])
    w.writeheader()
    w.writerows([{'PN': key, 'QTY': val} 
                 for key, val in pn_and_qty_no_duplicates.items()])

Import and prepare Part Numbers data

In [4]:
pn_and_qty = pd.read_csv(r'data/PN_and_QTY.csv')

Import and prepare data from 1C

In [5]:
data_from_1c = pd.read_csv(r'data/WOSS.csv')
data_from_1c.columns = ['Part no.'] + list(data_from_1c.columns[1:])
data_from_1c.columns

Index([u'Part no.', u'Serial no.', u'Part description', u'Segment', u'PO no.',
       u'PO date', u'PO line', u'PO qty', u'Invoice no.', u'Invoice value',
       u'Invoice qty', u'AU', u'AC', u'DOC', u'Point of departure',
       u'Requested point of arrival', u'Mode of transportation',
       u'DOC comment', u'DOC date', u'Green light requested',
       u'Green light received', u'Green light granted',
       u'Actual point of arrival', u'Waybills', u'Shipment comment',
       u'Pre alert', u'Estimated time of departure',
       u'Actual time of departure', u'Estimated time of arrival',
       u'Actual time of arrival', u'Dox received', u'In customs', u'Cleared',
       u'Customs file no.', u'Collected from port', u'Delivered to WH',
       u'Local distribution ETD', u'Local distribution ATD',
       u'Local distribution ETA', u'Local distribution ATA'],
      dtype='object')

In [54]:
data_from_1c = data_from_1c[[u'Part no.', u'Part description', u'PO no.', u'PO date', u'PO qty', 
                            u'Customs file no.', u'Collected from port']]
data_from_1c[u'Collected from port'] = pd.to_datetime(data_from_1c[u'Collected from port'], dayfirst=True)
data_from_1c['PO qty'] = pd.to_numeric(data_from_1c[u'PO qty'], errors='coerce')
data_from_1c = data_from_1c.dropna(subset=[u'Collected from port', u'PO qty'])

data_from_1c.sort_values(by=[u'Collected from port'], ascending=False, inplace=True)
data_from_1c.columns

Index([u'Part no.', u'Part description', u'PO no.', u'PO date', u'PO qty',
       u'Customs file no.', u'Collected from port'],
      dtype='object')

Import and prepare data from Store

In [9]:
data_from_store = pd.read_csv(r'data/STORE_01012014-30062014.csv', low_memory=False)
data_from_store.columns

Index([u'Part Number', u'Part Location', u'Transaction Id', u'Product Id',
       u'Source Store Id', u'Source Accounting Unit Id',
       u'Source Accounting Unit Name', u'Destination Accountint Unit Id',
       u'Destination Accounting Unit Name', u'Balance Qty To Be Received',
       u'Created Date', u'Extended Value', u'IMCS Classified',
       u'Last Purchased Price', u'Last Purchased Price Unit',
       u'Last Updated Date', u'Line Number', u'Line Number ERP',
       u'Source Location Id', u'Source Location', u'Destination Location Id',
       u'Destination Location', u'Part Type', u'Product',
       u'Product Description', u'Quantity Txn Part', u'Serialized Product',
       u'Sub Sub Segment Part Id', u'Destination Store Id',
       u'Sub Sub Segment Part Name', u'SWPS PO Number', u'SWPS PO Line Number',
       u'To Date Goods Received Qty', u'Transaction Primary Id', u'Txn Commit',
       u'Vendor Part Description', u'Vendor PN', u'Integration Code Product',
       u'Integratio

In [10]:
data_from_store = data_from_store[[u'Part Number', u'Transaction Id', u'Last Updated Date', u'Quantity Txn Part',
                                   u'Transaction Status Name']]
data_from_store = data_from_store[data_from_store[u'Transaction Status Name'] == 'Closed']
data_from_store[u'Last Updated Date'] = pd.to_datetime(data_from_store[u'Last Updated Date'])
data_from_store.sort_values(by=[u'Last Updated Date'], ascending=False, inplace=True)
data_from_store.columns

Index([u'Part Number', u'Transaction Id', u'Last Updated Date',
       u'Quantity Txn Part', u'Transaction Status Name'],
      dtype='object')

Process all Part Numbers and their quantities

In [13]:
pn_qty = {pn: {'QTY': qty} for pn, qty in itertools.izip(pn_and_qty['PN'], pn_and_qty['QTY'])}

In [64]:
pns = data_from_store[u'Part Number']
pn1c = data_from_1c[u'Part no.']

In [55]:
for pn, table in pn_qty.iteritems():
    pn_qty[pn]['storeinfo'] = data_from_store[0:0]
    pn_qty[pn]['1cstoreinfo'] = data_from_1c[0:0]
    pn_qty[pn]['1cinfo'] = data_from_1c[0:0]
    qty_left_store = qty_left_1c = table['QTY']
    table_store = data_from_store[data_from_store[u'Part Number'] == pn]
    table_1c = data_from_1c[data_from_1c[u'Part no.'] == pn]
    while qty_left_store > 0 and not table_store.empty:
        pn_qty[pn]['storeinfo'].append(table_store.iloc[0], ignore_index=True)
        qty_left_1c_store = table_store.iloc[0].at[u'Quantity Txn Part']
        qty_left_store -= qty_left_1c_store
        store_time = table_store.iloc[0].at[u'Last Updated Date']
        table_1c_store = table_1c[table_1c[u'Collected from port'] < store_time]
        while qty_left_1c_store > 0 and not table_1c_store.empty:
            pn_qty[pn]['1cstoreinfo'].append(table_1c_store.iloc[0], ignore_index=True)
            qty_recorded_in_1c_and_store = table_1c_store.iloc[0].at[u'PO qty']
            qty_left_1c -= qty_recorded_in_1c_and_store
            qty_left_1c_store -= qty_recorded_in_1c_and_store
            table_1c_store = table_1c_store[1:]
#     mask = [row in pn_qty[pn]['1cstoreinfo']]
#     table_1c = table_1c.where(table_1c not in pn_qty[pn]['1cstoreinfo'])
# #     while qty_left_1c > 0 and ta

KeyboardInterrupt: 

In [49]:
qty_recorded_in_1c_and_store

'6'

In [46]:
# qty_left_1c_store = table_store[0:1]
qty_left_1c_store
table_store.iloc[0].at[u'Quantity Txn Part']


AttributeError: 'numpy.float64' object has no attribute 'asfloat'

In [204]:
mask = [(row in pn_qty[pn]['1cstoreinfo']) for row in table_1c.iterrows()]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [211]:
table_1c.drop(pn_qty[pn]['1cstoreinfo'])

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [193]:
pn_qty[pn]['1cstoreinfo'].append(table_1c.ix[0])

In [197]:
pn_qty[pn]['1cstoreinfo']

[Part no.                                                         B012828
 Part description       O-RING SZ 230 2.484 IDX 0.139W VITON 95Duro_  ...
 PO no.                                                        WOSS08284Y
 PO date                                                       10.06.2014
 PO qty                                                               100
 Customs file no.                               10707090-210714-0006470  
 Collected from port                                  2014-07-23 00:00:00
 Name: 0, dtype: object]

In [44]:
pn = 'B012828'
table_store = data_from_store[data_from_store['Part Number'] == pn]
table_1c = data_from_1c[data_from_1c['PO no.'] == pn]
pn_qty[pn]['storeinfo'] = []
pn_qty[pn]['1cinfo'] = []

In [165]:
table_store = data_from_store.ix[pns == pn].reset_index(drop=True)

In [183]:
table_store.ix[5:]

,Part Number,Transaction Id,Last Updated Date,Quantity Txn Part,Transaction Status Name


In [150]:
pn_qty[pn]['storeinfo'].append(table_store.ix[0])

In [151]:
qty_left_1c_store = table_store.get_value(0, u'Quantity Txn Part')

In [154]:
store_time

Timestamp('2015-09-25 02:15:00')

In [181]:
pn_qty

{'0013222': {'QTY': 2.0},
 '0013243': {'QTY': 2.0},
 '0013246': {'QTY': 2.0},
 '0013250': {'QTY': 15.0},
 '0028724': {'QTY': 3.0},
 '0078407': {'QTY': 12.0},
 '100044690': {'QTY': 2.0},
 '100083359': {'QTY': 3.0},
 '100083361': {'QTY': 4.0},
 '100083362': {'QTY': 3.0},
 '100119266': {'QTY': 40.0},
 '100131400': {'QTY': 1.0},
 '100131402': {'QTY': 1.0},
 '100135812': {'QTY': 2.0},
 '100150240': {'QTY': 0.0},
 '100151456': {'QTY': 6.0},
 '100152630': {'QTY': 2.0},
 '100195982': {'QTY': 6.0},
 '100233958': {'QTY': 7.0},
 '100234017': {'QTY': 1.0},
 '100246073': {'QTY': 15.0},
 '100247501': {'QTY': 3.0},
 '100247507': {'QTY': 3.0},
 '100272458': {'QTY': 2.0},
 '100289461': {'QTY': 2.0},
 '100313951': {'QTY': 0.0},
 '100320987': {'QTY': 17.0},
 '100323973': {'QTY': 2.0},
 '100333872': {'QTY': 2.0},
 '100350783': {'QTY': 16.0},
 '100369186': {'QTY': 4.0},
 '100369188': {'QTY': 2.0},
 '100369189': {'QTY': 2.0},
 '100387485': {'QTY': 1.0},
 '100387486': {'QTY': 2.0},
 '100387487': {'QTY': 2.0}

In [153]:
store_time = table_store.get_value(0, u'Last Updated Date')

In [155]:
table_1c[u'Collected from port'] < store_time

6931    True
6893    True
6637    True
5264    True
5263    True
4655    True
4328    True
3117    True
Name: Collected from port, dtype: bool

In [158]:
table_1c = table_1c.where(table_1c[u'Collected from port'] < store_time)

In [163]:
table_1c.reset_index(drop=True)

,Part no.,Part description,PO no.,PO date,PO qty,Customs file no.,Collected from port
0,B012828,O-RING SZ 230 2.484 IDX 0.139W VITON 95Duro_ ...,WOSS08284Y,10.06.2014,100,10707090-210714-0006470,2014-07-23
1,B012828,O-RING SZ 230 2.484 IDX 0.139W VITON 95Duro_ ...,WOSS08241W,29.05.2014,500,10707090-150714-0006191,2014-07-17
2,B012828,O-RING SZ 230 2.484 IDX 0.139W VITON 95Duro_ ...,WOSS08179X,21.05.2014,500,10707090-040714-0005806,2014-07-08
3,B012828,O-RING SZ 230 2.484 IDX 0.139W VITON 95Duro_ ...,WOSS07806V,05.11.2013,300,10707090-100114-0000070,2014-01-13
4,B012828,O-RING SZ 230 2.484 IDX 0.139W VITON 95Duro_ ...,WOSS07806V,05.11.2013,300,10707090-171213-0010713,2013-12-19
5,B012828,O-RING SZ 230 2.484 IDX 0.139W VITON 95Duro_ ...,WOSS07717U,18.09.2013,400,10707090-121113-0009528,2013-11-14
6,B012828,O-RING SZ 2-230 VITON 95D ...,WOSS07619X,26.07.2013,500,10707090-050913-0007534,2013-09-05
7,B012828,O-RING SZ 2-230 VITON 95D Or...,WOSS07110Y,22.08.2012,1 000,10707090-091012-0009765,2012-10-11


In [37]:
counter = collections.Counter(data_from_1c[u'Part no.'])

In [38]:
pn_counter = {}
for pn in pn_qty.iterkeys():
    pn_counter[pn] = counter[pn]

In [39]:
pn_counter

{'0013222': 0,
 '0013243': 0,
 '0013246': 0,
 '0013250': 0,
 '0028724': 0,
 '0078407': 2,
 '100044690': 0,
 '100083359': 7,
 '100083361': 6,
 '100083362': 6,
 '100119266': 5,
 '100131400': 2,
 '100131402': 3,
 '100135812': 0,
 '100150240': 10,
 '100151456': 7,
 '100152630': 1,
 '100195982': 6,
 '100233958': 2,
 '100234017': 1,
 '100246073': 6,
 '100247501': 11,
 '100247507': 5,
 '100272458': 10,
 '100289461': 0,
 '100313951': 0,
 '100320987': 5,
 '100323973': 3,
 '100333872': 0,
 '100350783': 0,
 '100369186': 5,
 '100369188': 0,
 '100369189': 32,
 '100387485': 1,
 '100387486': 2,
 '100387487': 1,
 '100427756': 4,
 '100430470': 5,
 '100453600': 0,
 '100459169': 0,
 '100459196': 2,
 '100471948': 0,
 '100471959': 2,
 '100545287': 0,
 '100546138': 0,
 '100547767': 2,
 '100593948': 3,
 '100596370': 1,
 '100600474': 0,
 '100629933': 1,
 '100672274': 3,
 '100735453': 4,
 '100756246': 5,
 '100777673': 0,
 '100809961': 11,
 '100885913': 1,
 '100886692': 0,
 '100886694': 1,
 '100987137': 6,
 '10

In [113]:
pn_and_qty.columns

Index([u'PN', u'QTY'], dtype='object')

In [117]:
pn_and_qty[[u'PN', u'QTY']]

,PN,QTY
0,T6005108,2
1,A077468,4
2,H222341,10
3,100987137,4
4,P215562,1
5,P215563,1
6,P125975,6
7,H441236,2
8,P017957,4
9,100195982,6


In [39]:
list(pn_and_qty.iteritems())

[('PN', 0        5047097
  1        6907805
  2      100044690
  3      100083359
  4      100083361
  5      100083362
  6      100119266
  7      100119266
  8      100119266
  9      100131400
  10     100131402
  11     100135812
  12     100150240
  13     100151456
  14     100152630
  15     100195982
  16     100233958
  17     100233958
  18     100234017
  19     100246073
  20     100246073
  21     100246073
  22     100246073
  23     100246073
  24     100247501
  25     100247507
  26     100272458
  27     100289461
  28     100313951
  29     100320987
           ...    
  526      P189654
  527      P191838
  528      P191845
  529      P191971
  530      P192017
  531      P195951
  532      P215562
  533      P215563
  534      P286568
  535      P329067
  536      P334864
  537      P495588
  538      P500414
  539      P676065
  540      P694786
  541      P792910
  542     T6005013
  543     T6005014
  544     T6005108
  545     T6005134
  546     T6005172
  547 

In [178]:
table = []
for pn in pn_qty.iterkeys():
    table.append(data_from_1c.where(data_from_1c['Part no.'] == pn))

In [180]:
table[0]

,Part no.,Part description,PO no.,PO date,PO qty,Customs file no.,Collected from port
4133,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7231,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7246,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7245,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7244,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7243,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7242,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7241,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7240,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7239,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [172]:
type(table)

pandas.core.frame.DataFrame